In [208]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from datetime import datetime
import numpy as np

### This file is mainly for ease of testing new alterations to the graph

### Ways to filter the display of the graph
# Set FOCUS_MATCH to a single match name - shows competing teams and associated players for that match only
# Set TEAM_NODE_A and TEAM_NODE_B to two teams you want to see neighborhood connections of
#     Optionally cap the number of edges to show with MAX_EVENT_DISPLAY
# Set TIMESTAMP_START and TIMESTAMP_END to display all nodes and edges in a certain time window

# FOCUS_MATCH = 'ESPORTSTMNT01/1641087' # Match to focus on
FOCUS_MATCH = None
MAX_EVENT_DISPLAY = 120 # Custom number of neighbors to display; not impactful if FOCUS_MATCH set
# MAX_EVENT_DISPLAY = None
TEAM_NODE_A = 1740 # Home team node to focus on and see neighbors of
TEAM_NODE_B = 1455 # Away team node to focus on and see neighbors of
# NODE_ADJ = 10240 # Single node for node adjacency method
# Earliest timestamp: 2014-01-14 17:52:02
# Latest timestamp: 2023-11-20 20:40:26
# TIMESTAMP_START = np.datetime64('2023-11-20 19:59:21')
# TIMESTAMP_END = np.datetime64('2023-11-20 19:59:22')
TIMESTAMP_START = None
TIMESTAMP_END = None
COMBO_EDGES = True


In [209]:
df = pd.read_csv('data/processed/lol/events_with_gameid.csv')
dfLolTeams = pd.read_csv('data/processed/lol/teams_with_names.csv')
dfLolPlayers = pd.read_csv('data/processed/lol/players_with_names.csv')

df = pd.merge(df, dfLolTeams[['team_num', 'teamname']], how='left', left_on='u', right_on='team_num')
df = df.rename(columns={'teamname': 'u_name'})
df = df.drop('team_num', axis=1)
df_1 = df[df['v_type'] == 1]
df_2 = df[df['v_type'] == 2]

# Get team names in for v
df_1 = pd.merge(df_1, dfLolTeams[['team_num', 'teamname']], how='left', left_on='v', right_on='team_num')
df_1 = df_1.drop('team_num', axis=1)
df_1 = df_1.rename(columns={'teamname': 'v_name'})
# Get player names in for v
df_2 = pd.merge(df_2, dfLolPlayers[['player_num', 'playername']], how='left', left_on='v', right_on='player_num')
df_2 = df_2.drop('player_num', axis=1)
df_2 = df_2.rename(columns={'playername': 'v_name'})

df_recombined = pd.concat([df_1, df_2])
df = df_recombined.sort_values('e_idx')
df['ts'] = pd.to_datetime(df['ts'], unit='s')

# Print earliest and latest timestamps for teams A and B
if TEAM_NODE_A and TEAM_NODE_B:
    teams_df = df.loc[(df['u'] == TEAM_NODE_B) | (df['v'] == TEAM_NODE_B)]
    print('TEAM A AND B\'S EARLIEST TIMESTAMP: ' + str(teams_df['ts'].min()))
    print('                    MAX TIMESTAMP: ' + str(teams_df['ts'].max()))

# Time window selection
if TIMESTAMP_START and TIMESTAMP_END:
    df = df.loc[(df['ts'] >= TIMESTAMP_START) & (df['ts'] <= TIMESTAMP_END)]
    print('SELECTING TIME WINDOW BETWEEN ' + str(TIMESTAMP_START) + ' AND ' + str(TIMESTAMP_END))

# dfLolTeams.sort_values('teamname')
# df.loc[(df['v'] == 1455) & (df['e_type'].isin([1,2]))]

TEAM A AND B'S EARLIEST TIMESTAMP: 2020-11-28 01:46:59
                    MAX TIMESTAMP: 2023-11-18 08:04:29


In [210]:
if TEAM_NODE_A and TEAM_NODE_B:
    dfGames = df.loc[(df['u'].isin([TEAM_NODE_A, TEAM_NODE_B])) & (df['v'].isin([TEAM_NODE_A, TEAM_NODE_B])) & (df['e_type'].isin([1, 2]))] # should be sorted by date
    print('TEAM A (' + str(TEAM_NODE_A) + ') AND TEAM B (' + str(TEAM_NODE_B) + ') PLAYED EACH OTHER IN THESE ' + str(len(dfGames)) + ' MATCHES:')
    listGames = list(dfGames[['u', 'v', 'ts', 'gameid']].itertuples(index=False))
    for game in listGames:
        print('Home (' + str(game[0]) + '); Away (' + str(game[1]) + ') --- ' + str(game[2]) + ' --- ' + game[3])

# Enable to display a single match
if FOCUS_MATCH:
    df = df.loc[(df['gameid'] == FOCUS_MATCH)]
    df

TEAM A (1740) AND TEAM B (1455) PLAYED EACH OTHER IN THESE 21 MATCHES:
Home (1740); Away (1455) --- 2020-12-20 04:42:50 --- ESPORTSTMNT01/1641069
Home (1740); Away (1455) --- 2020-12-20 05:49:03 --- ESPORTSTMNT01/1641087
Home (1455); Away (1740) --- 2020-12-20 06:31:38 --- ESPORTSTMNT01/1651205
Home (1455); Away (1740) --- 2020-12-20 07:22:05 --- ESPORTSTMNT01/1651206
Home (1740); Away (1455) --- 2020-12-20 08:06:02 --- ESPORTSTMNT01/1641090
Home (1455); Away (1740) --- 2022-04-30 04:49:03 --- ESPORTSTMNT05_2670220
Home (1740); Away (1455) --- 2022-04-30 05:38:19 --- ESPORTSTMNT05_2670222
Home (1740); Away (1455) --- 2022-05-08 06:18:25 --- ESPORTSTMNT05_2680479
Home (1455); Away (1740) --- 2022-05-08 07:35:18 --- ESPORTSTMNT05_2680486
Home (1455); Away (1740) --- 2022-10-22 04:15:54 --- ESPORTSTMNT02_3048543
Home (1740); Away (1455) --- 2022-10-22 05:39:55 --- ESPORTSTMNT02_3048601
Home (1740); Away (1455) --- 2022-10-30 06:50:33 --- ESPORTSTMNT02_3049517
Home (1455); Away (1740) --- 

In [202]:
# Utility cell
# df_events = pd.read_csv('data/processed/lol/events_with_gameid.csv')
# p6746 = df_events.loc[(df_events['u'] == 1740) & (df_events['v'] == 6746) & (df_events['e_type'] == 4) & (df_events['gameid'] == 'ESPORTSTMNT01/1641087')]
# p6746

In [273]:
focusNodes = [ # Uncomment to display 2 teams method
    {
        'name': TEAM_NODE_A,
        'nodeList': [],
        'edgeList': []
    },
    {
        'name': TEAM_NODE_B,
        'nodeList': [],
        'edgeList': []
    }
]
# focusNodes = [ # Uncomment to display single node adjacency method
#     {
#         'name': NODE_ADJ,
#         'nodeList': [],
#         'edgeList': []
#     }
# ]

nodeLabelMapper = {
    1: '  Team:  \n',
    2: '  Player:  \n'
}

edgeLabelMapper = {
    1: 'Lost',
    2: 'Won',
    3: 'Joined',
    4: 'Info'
}

# Keys the same as node type
# (1) team
# (2) player
nodeSizeMapper = {
    1: 30,
    2: 15
}

nodeColorMapper = {
    1: '#000099',
    2: '#0099FF'
}

# Keys the same as edge_type
# (1) Lost
# (2) Won
# (3) Played
# (4) Game info
edgeColorMapper = {
    1: '#FF0000',
    2: '#15B01A',
    3: '#0099FF',
    4: '#C0C0C0'
}

edgeFontSizeMapper = {
    1: 24,
    2: 24,
    3: 12,
    4: 10
}

def edgeWeightMapper(type, tsList):
    if type == 1 or type == 2:
        return len(tsList) * 3
    elif type == 3:
        return 1 + (len(tsList) % 10)
    return 1

def reformatLabel(text):
    return text.replace(' ', '\n')

def fontAdjuster(type, text):
    if type == 1:
        return 24 if len(text) < 20 else 20
    else:
        return 18 if len(text) < 20 else 14
    
def edgeTitleMaker(tsList):
    tsList = pd.to_datetime(tsList).strftime('%h %d %Y %H:%M:%S').tolist()
    concat = '\n'.join(str(ts) for ts in tsList)
    return concat

for focusNode in focusNodes:
    selectedTeam = df.loc[ # Uncomment for 2 teams method, and if adj method is for a team
        (df['u'] == focusNode['name']) | ((df['v'] == focusNode['name']) & (df['v_type'] == 1)),
        ['u', 'v', 'u_type', 'v_type', 'e_type', 'u_name', 'v_name', 'ts']]
    # selectedTeam = df.loc[ # Uncomment for adj method, if for a player
    #     (df['v'] == focusNode['name']) & (df['v_type'] == 2),
    #     ['u', 'v', 'u_type', 'v_type', 'e_type', 'u_name', 'v_name', 'ts']]
    if MAX_EVENT_DISPLAY:
        selectedTeam = selectedTeam.head(MAX_EVENT_DISPLAY)
    teamTypeDF = selectedTeam.drop_duplicates()
    us = list(teamTypeDF[['u', 'u_type', 'u_name', 'ts']].itertuples(index=False, name=None))
    vs = list(teamTypeDF[['v', 'v_type', 'v_name', 'ts']].itertuples(index=False, name=None))
    nodesList = list(list(dict.fromkeys(us + vs)))
    nodesList = list(map(lambda x: (x[0], {
        'type': x[1],
        'label': reformatLabel(x[2]),
        'title': 'TODO',
        'margin': 15,
        'color': {
            'background': nodeColorMapper[x[1]], 
            'highlight': {
                'background': nodeColorMapper[x[1]],
                'border': 'magenta'
            },
            'hover': {
                'border': 'gray'
            }
        },
        # 'physics': False,
        # 'mass': 8 if x[1] == 1 else 2,
        'mass': 5,
        'shape': 'circle',
        'font': {'size': fontAdjuster(x[1], x[2]), 'color': 'white'},
        'borderWidthSelected': 6
        }), nodesList))
    edgesWithDups = selectedTeam.groupby(selectedTeam.columns.tolist(), as_index=False).size()
    edgesNoDupsDF = edgesWithDups.groupby(['u', 'v', 'e_type'])
    edgesNoDupsList = []
    for ed, group in edgesNoDupsDF:
        edgesNoDupsList.append(ed + (group['ts'].tolist(),))
    edgesWithDupsList = list(edgesWithDups[['u', 'v', 'e_type', 'size', 'ts']].itertuples(index=False, name=None))
    # Change arrow display direction if edge type is 3 (to show player joining team):
    edgesWithDupsList = list(map(lambda x: (x[1] if x[2] == 3 else x[0], x[0] if x[2] == 3 else x[1], { 
        'edge_type': x[2], 
        # 'weight': edgeWeightMapper(x[2], x[3]),
        'weight': 1,
        'label': edgeLabelMapper[x[2]],
        'title': str(pd.to_datetime(str(x[4])).strftime('%h %d %Y %H:%M:%S')),
        # 'title': 'dingus',
        'color': edgeColorMapper[x[2]],
        'font': {'size': edgeFontSizeMapper[x[2]]},
        'smooth': True,
        # 'arrowSize': 5,
        # 'physics': False
        }), edgesWithDupsList))
    edgesNoDupsList = list(map(lambda x: (int(x[1] if x[2] == 3 else x[0]), int(x[0] if x[2] == 3 else x[1]), { 
        'edge_type': int(x[2]), 
        'weight': edgeWeightMapper(x[2], x[3]),
        'label': edgeLabelMapper[x[2]],
        'title': edgeTitleMaker(x[3]),
        'color': edgeColorMapper[x[2]],
        'font': {'size': edgeFontSizeMapper[x[2]]},
        'smooth': 0,
        # 'arrowSize': 5,
        # 'physics': False
        }), edgesNoDupsList))
    focusNode['nodeList'] = nodesList
    # focusNode['edgeList'] = edgesWithDupsList
    focusNode['edgeList'] = edgesNoDupsList

edgesNoDupsList
<class 'int'>
<class 'int'>
edgesNoDupsList
<class 'int'>
<class 'int'>


In [276]:
focusGraph = nx.MultiDiGraph()

lineWidthMapper = {
    1: 6,
    2: 6,
    3: 3,
    4: 1
}

arrowSizeMapper = {
    1: 10,
    2: 40,
    3: 12,
    4: 1
}

allNodesList = focusNodes[0]['nodeList'] + focusNodes[1]['nodeList']
allEdgesList = focusNodes[0]['edgeList'] + focusNodes[1]['edgeList']

focusGraph.add_nodes_from(allNodesList)
focusGraph.add_edges_from(allEdgesList)

options = {
    'arrowstyle': '->',
    'arrowsize': list(arrowSizeMapper[edge_type] for u, v, edge_type in list(focusGraph.edges(data='edge_type')))
}

from pyvis.network import Network
import pyvis

net = Network(
    '1500px', '1500px',
    directed=True,
    # heading='League of Legends',
    # select_menu=True,
    # filter_menu=True,
    # bgcolor='#222222',
    # font_color='white'
)
# net.toggle_physics(False)
# net.repulsion()
net.from_nx(focusGraph) # Create directly from nx graph

options = {
    'physics':{ # physics very distracting with large/non-capped number of edges, even with just 2 teams
        'barnesHut':{
            'gravitationalConstant': -15000, # seemingly best around -15000
            'centralGravity': 5, # seemingly best at 5
            'springLength': 300,
            'springConstant': 0.1,
            'damping': 3,
            'avoidOverlap': 25 # higher vals push nodes away from each other actively
        }
    },
    'interaction':{   
        'selectConnectedEdges': True,
        'hover': True
    },
    'edges': {
        'arrowStrikethrough': False
    }
}

net.options=options
# net.toggle_physics(False)
net.show('test1.html', notebook=False, ) # do NOT remove the notebook=False

[(1455, 1740, {'edge_type': 1, 'weight': 6, 'label': 'Lost', 'title': 'Dec 20 2020 06:31:38\nApr 30 2022 04:49:03', 'color': '#FF0000', 'font': {'size': 24}, 'smooth': 0}), (1455, 1740, {'edge_type': 2, 'weight': 3, 'label': 'Won', 'title': 'Dec 20 2020 07:22:05', 'color': '#15B01A', 'font': {'size': 24}, 'smooth': 0}), (1455, 1740, {'edge_type': 4, 'weight': 1, 'label': 'Info', 'title': 'Dec 20 2020 07:04:36\nDec 20 2020 07:44:52\nApr 30 2022 05:16:10', 'color': '#C0C0C0', 'font': {'size': 10}, 'smooth': 0}), (1537, 1740, {'edge_type': 2, 'weight': 3, 'label': 'Won', 'title': 'May 01 2022 04:33:11', 'color': '#15B01A', 'font': {'size': 24}, 'smooth': 0}), (1537, 1740, {'edge_type': 4, 'weight': 1, 'label': 'Info', 'title': 'May 01 2022 04:59:59', 'color': '#C0C0C0', 'font': {'size': 10}, 'smooth': 0}), (1563, 1740, {'edge_type': 1, 'weight': 3, 'label': 'Lost', 'title': 'Apr 30 2022 06:32:13', 'color': '#FF0000', 'font': {'size': 24}, 'smooth': 0}), (1563, 1740, {'edge_type': 4, 'weig

In [ ]:
# from pyvis.network import Network

# net = Network(
#     '1500px', '1500px',
#     directed=True
# )
# net.from_nx(focusGraph) # Create directly from nx graph

# net.show('test1.html', notebook=False) # do NOT remove the notebook=False